# Movie Weekend Trip Project

## TODO
Write intro

## Retrieving the Data
This data comes from the [Shakespeare plays page on Kaggle](https://www.kaggle.com/kingburrito666/shakespeare-plays#Shakespeare_data.csv). We will begin by loading the dataset into a dataframe:

In [1]:
import numpy as np
import pandas as pd